# **PostgreSQL ile performans testi deneyimi**

<div style="text-align:center;"><img src="img/postgre_exp_erd.png" /></div>

<h2>PostgreSQL bağlantısı</h2>

**config.py** dosyası çalıştırılarak PostgreSQL veritabanına bağlantı için gerekli konfigürasyonlar yapılır.

In [2]:
%run -i "get_config.py"

<h2>Index ile performans testi</h2>

<h3>1 - Normal sorgu</h3>

Herhangi bir performans iyileştirmesi (index, partition, vs.) uygulanmadan sorgu süreleri test edilir.

In [3]:
ilk = datetime.now()

sorgu_index_normal = """
                    select * 
                    from Orders
                    where CustomerID in (
                        select CustomerID
                        from Customers
                        where Phone = '5318139006'
                    )
"""

cursor.execute(sorgu_index_normal)

son = datetime.now()

sure_index_normal = son - ilk

print("Normal çalışma süresi: ", sure_index_normal)

Normal çalışma süresi:  0:00:14.669950


<h3> 2 - CustomerID ve Phone üzerinde index oluşturulmuş sorgu</h3>

**Orders** ve **Customer** tablolarının birer kopyası oluşturulur (Orders_Index, Customers_Index):

<br />

``` sql
drop table if exists Customers_Index;
create table Customers_Index as select * from customers;

drop table if exists Orders_Index;
create table Orders_Index as select * from orders;
```

<br />

Orders_Index tablosu üzerinde **CustomerID** kolonu üzerinde index tanımlanır:

<br />

``` sql
drop index if exists idx_order_customerId;
create index idx_order_customerId on Orders_Index (CustomerID);
```

<br />

Customers_Index tablosu üzerinde **CustomerID** ve **Phone** kolonları üzerinde index tanımlanır:

<br />

``` sql
drop index if exists idx_customer_customerId;
create index idx_customer_customerId on Customers_Index (CustomerID);

drop index if exists idx_customer_phone;
create index idx_customer_phone on Customers_Index (Phone);
```

<br />

In [4]:
ilk = datetime.now()

sorgu_index_hizli = """
                        select * 
                        from Orders_Index
                        where CustomerID in (
                            select CustomerID
                            from Customers_Index
                            where Phone = '5318139006'
                        )
"""

cursor.execute(sorgu_index_hizli)

son = datetime.now()

sure_index_hizli = son - ilk

print("Index ile çalışma süresi: ", sure_index_hizli)

Index ile çalışma süresi:  0:00:00.138004


<h2>Partition ile performans testi</h2>

<h3>1 - Normal Sorgu</h3>

Herhangi bir performans iyileştirmesi (index, partition, vs.) uygulanmadan sorgu süreleri test edilir.

In [5]:
ilk = datetime.now()

sorgu_partition_normal = """
                    select p.Category, count(o.OrderID) as total
                    from orders o
                    left join products p
                        on o.ProductID = p.ProductID
                    where o.OrderDate = '2022-06-22'
                    group by p.Category
"""

cursor.execute(sorgu_partition_normal)

son = datetime.now()

sure_partition_normal = son - ilk

print("Normal çalışma süresi: ", sure_partition_normal)

Normal çalışma süresi:  0:00:07.826562


<h3>2 - OrderDate üzerinde aya göre partition oluşturulmuş sorgu</h3>

**Orders** tablosunun bir kopyası oluşturulur (Orders_Partition):

<br />

``` sql
create table Orders_Partition
(
  OrderID numeric(10),
  ProductID numeric(10),
  CustomerID numeric(10),
  Quantity int,
  OrderDate date
) partition by range (OrderDate)
;
```
<br />

Bu tablo üzerinde **OrderDate** kolonu üzerinde aya göre, 12 tane partition tanımlanır:

<br />

``` sql
create table order_part_2022_1 partition OF orders_partition for values from ('2022-01-01') to ('2022-02-01');
create table order_part_2022_2 partition OF orders_partition for values from ('2022-02-01') to ('2022-03-01');
create table order_part_2022_3 partition OF orders_partition for values from ('2022-03-01') to ('2022-04-01');
create table order_part_2022_4 partition OF orders_partition for values from ('2022-04-01') to ('2022-05-01');
create table order_part_2022_5 partition OF orders_partition for values from ('2022-05-01') to ('2022-06-01');
create table order_part_2022_6 partition OF orders_partition for values from ('2022-06-01') to ('2022-07-01');
create table order_part_2022_7 partition OF orders_partition for values from ('2022-07-01') to ('2022-08-01');
create table order_part_2022_8 partition OF orders_partition for values from ('2022-08-01') to ('2022-09-01');
create table order_part_2022_9 partition OF orders_partition for values from ('2022-09-01') to ('2022-10-01');
create table order_part_2022_10 partition OF orders_partition for values from ('2022-10-01') to ('2022-11-01');
create table order_part_2022_11 partition OF orders_partition for values from ('2022-11-01') to ('2022-12-01');
create table order_part_2022_12 partition OF orders_partition for values from ('2022-12-01') to ('2023-01-01');
```

<br />

Daha sonra ilgili aylar her partition'a insert edilir:

<br />

``` sql
insert into Orders_Partition select * from orders where OrderDate >= '2022-01-01' and OrderDate < '2022-02-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-02-01' and OrderDate < '2022-03-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-03-01' and OrderDate < '2022-04-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-04-01' and OrderDate < '2022-05-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-05-01' and OrderDate < '2022-06-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-06-01' and OrderDate < '2022-07-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-07-01' and OrderDate < '2022-08-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-08-01' and OrderDate < '2022-09-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-09-01' and OrderDate < '2022-10-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-010-01' and OrderDate < '2022-11-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-011-01' and OrderDate < '2022-12-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-012-01' and OrderDate < '2023-01-01';
```

<br />

In [6]:
ilk = datetime.now()

sorgu_partition_hizli = """
                    select p.Category, count(o.OrderID) as total
                    from Orders_Partition o
                    left join products p
                        on o.ProductID = p.ProductID
                    where o.OrderDate = '2022-06-22'
                    group by p.Category
"""

cursor.execute(sorgu_partition_hizli)

son = datetime.now()

sure_partition_hizli = son - ilk

print("Partition ile çalışma süresi: ", sure_partition_hizli)

Partition ile çalışma süresi:  0:00:00.828229


<h3>Karşılaştırma</h3>

In [7]:
print(" Index Test", "\n", "-"*20)
print("Normal\t\t", sure_index_normal)
print("Index\t\t", sure_index_hizli)
print("Hız artışı (%)\t", ((sure_index_normal - sure_index_hizli) / sure_index_normal) * 100)
print("\n")
print(" Partition Test", "\n", "-"*20)
print("Normal\t\t", sure_partition_normal)
print("Partition\t", sure_partition_hizli)
print("Hız artışı (%)\t", ((sure_partition_normal - sure_partition_hizli) / sure_partition_normal) * 100)

 Index Test 
 --------------------
Normal		 0:00:14.669950
Index		 0:00:00.138004
Hız artışı (%)	 99.05927423065518


 Partition Test 
 --------------------
Normal		 0:00:07.826562
Partition	 0:00:00.828229
Hız artışı (%)	 89.41771623351352
